In [15]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import os
import heapq
import random
import numpy as np
import pickle
import io

from faceRecognizer import FaceRecognizer

class JupyterFaceRatingApp:
    def __init__(self, image_paths):
        """ Initialize the face rating app for Jupyter notebook. """
        
        self.face_recognizer = FaceRecognizer('models/encoderModel.dat', 'models/recognitionModel.dat')
        
        # Shuffle and prepare image paths
        self.image_paths = image_paths.copy()
        random.shuffle(self.image_paths)
        
        # Initialize the heap and lists for liked and disliked images
        self.image_heap = []
        self.liked_images = []
        self.disliked_images = []
        self.create_image_heap()
        self.current_image_path = None
        
        # Precompute facial descriptors for all images if needed
        print("Loading facial descriptors...")
        self.face_descriptors = self.load_or_compute_descriptors()
        print(f"Loaded descriptors for {len(self.face_descriptors)} images")
        
        # Init the UI
        self.setup_ui()
        
        # Show the first image
        self.next_image()
    
    def setup_ui(self):
        """ Setup the interactive widgets for Jupyter notebook. """
        
        # widget to display images
        self.image_widget = widgets.Image(
            value=b'',
            format='png',
            width=300,
            height=300
        )
        # banner text
        self.status_text = widgets.HTML(
            value="<h3>Like or Pass on the Image!</h3>",
            layout=widgets.Layout(text_align='center')
        )
        # show database statistics
        self.stats_text = widgets.HTML(
            value="<p>Liked: 0 | Passed: 0 | Remaining: 0</p>",
            layout=widgets.Layout(text_align='center')
        )
        # like Button
        self.like_btn = widgets.Button(
            description='❤️ Like',
            button_style='success',
            layout=widgets.Layout(width='120px', height='50px'),
            style={'font_size': '14px', 'font_weight': 'bold'}
        )
        # pass Button
        self.pass_btn = widgets.Button(
            description='❌ Pass',
            button_style='danger',
            layout=widgets.Layout(width='120px', height='50px'),
            style={'font_size': '14px', 'font_weight': 'bold'}
        )
        
        # button callbacks
        self.like_btn.on_click(self.on_like_clicked)
        self.pass_btn.on_click(self.on_pass_clicked)
        
        # button container
        self.button_box = widgets.HBox(
            [self.like_btn, self.pass_btn],
            layout=widgets.Layout(justify_content='center', margin='20px 0')
        )
        
        # progress bar
        self.progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=len(self.image_paths),
            description='Progress:',
            bar_style='info',
            orientation='horizontal'
        )
        
        # main container
        self.main_container = widgets.VBox([
            self.status_text,
            self.image_widget,
            self.stats_text,
            self.button_box,
            self.progress_bar
        ], layout=widgets.Layout(align_items='center'))
    
    def display(self):
        """ Display the app. """
        display(self.main_container)
    
    def pil_to_widget_image(self, pil_image):
        """ Convert PIL image to bytes for widget. """
        # resize image for display
        pil_image = pil_image.resize((300, 300), Image.Resampling.LANCZOS)
        
        # convert to bytes for ipywidgets
        img_buffer = io.BytesIO()
        pil_image.save(img_buffer, format='PNG')
        img_bytes = img_buffer.getvalue()
        
        return img_bytes
    
    def show_current_image(self):
        """ Display the current image in the widget. """
        if self.current_image_path:
            try:
                # Load and convert image
                img = Image.open(self.current_image_path)
                img_bytes = self.pil_to_widget_image(img)
                
                # update widget
                self.image_widget.value = img_bytes
                
                # update image label
                filename = os.path.basename(self.current_image_path)
                self.status_text.value = f"<h3>Like or Pass on the Image!</h3><p><i>{filename}</i></p>"
                
            except Exception as e:
                print(f"Error displaying image {self.current_image_path}: {e}")
                self.next_image()
    
    def update_stats(self):
        """ Update the statistics display. """
        liked_count = len(self.liked_images)
        passed_count = len(self.disliked_images)
        remaining_count = len(self.image_heap)
        total_evaluated = liked_count + passed_count
        # update stats text
        self.stats_text.value = f"""
        <p><strong>Liked:</strong> {liked_count} | 
        <strong>Passed:</strong> {passed_count} | 
        <strong>Remaining:</strong> {remaining_count}</p>
        """
        
        # update progress bar
        self.progress_bar.value = total_evaluated
        self.progress_bar.description = f'Progress: {total_evaluated}/{len(self.image_paths)}'
    
    def on_like_clicked(self, button):
        """ Handle like button click. """
        if self.current_image_path:
            print(f"Liked: {self.current_image_path}")
            self.liked_images.append(self.current_image_path)
            # Update weights for similar images in queue
            self.update_weights(self.current_image_path, "like")
            self.next_image()
    
    def on_pass_clicked(self, button):
        """ Handle pass button click. """
        if self.current_image_path:
            print(f"Passed: {self.current_image_path}")
            self.disliked_images.append(self.current_image_path)
            # Update weights for similar images in queue 
            self.update_weights(self.current_image_path, "pass")
            self.next_image()
    
    def next_image(self):
        """ Move to the next image in the heap. """
        if self.image_heap:
            # Get the next image with the highest priority (lowest weight)
            next_image = heapq.heappop(self.image_heap)
            # Get path from 3-tuple (weight, index, path)
            self.current_image_path = next_image[2]
            self.show_current_image()
            self.update_stats()
        else:
            self.finish_rating()
    
    def finish_rating(self):
        """ Handle completion of rating session. """
        # It would be pretty tough to rate all, there are MANY
        self.image_widget.value = b''
        self.status_text.value = "<h2>🎉 Rating Complete!</h2>"
        
        # Show final statistics
        liked_count = len(self.liked_images)
        passed_count = len(self.disliked_images)
        total = liked_count + passed_count
        
        if total > 0:
            like_percentage = (liked_count / total) * 100
            results_html = f"""
            <div style='text-align: center; padding: 20px;'>
                <h3>Final Results</h3>
                <p><strong>Total Rated:</strong> {total}</p>
                <p><strong>Liked:</strong> {liked_count} ({like_percentage:.1f}%)</p>
                <p><strong>Passed:</strong> {passed_count} ({100-like_percentage:.1f}%)</p>
            </div>
            """
        else:
            results_html = "<p>No images were rated.</p>"
        
        self.stats_text.value = results_html
        
        # disable buttons
        self.like_btn.disabled = True
        self.pass_btn.disabled = True
    
    def update_weights(self, image_path, action, k=5):
        """ Update the weights of similar images based on user feedback."""
        similar_images = self.get_similar_images(image_path, k)
        
        # Create a new heap with updated weights
        new_heap = []
        for weight, index, path in self.image_heap:
            if path in similar_images:
                # Lower weight = higher priority
                if action == "like":
                    new_weight = weight - 1  
                # Higher weight = lower priority
                elif action == "pass":
                    new_weight = weight + 1  
                else:
                    new_weight = weight
                heapq.heappush(new_heap, (new_weight, index, path))
            else:
                heapq.heappush(new_heap, (weight, index, path))
        
        self.image_heap = new_heap
        
        if similar_images:
            print(f"Updated weights for {len(similar_images)} similar images ({action})")
    
    def load_or_compute_descriptors(self):
        """ Load descriptors from cache or compute them if cache doesn't exist. """
        cache_file = "face_descriptors_cache.pkl"
        
        # Check if cache exists
        if os.path.exists(cache_file):
            try:
                print("Loading descriptors from cache...")
                with open(cache_file, 'rb') as f:
                    cached_descriptors = pickle.load(f)
                    
                    # Verify all current images are in cache
                    if all(img_path in cached_descriptors for img_path in self.image_paths):
                        print(f"Successfully loaded {len(cached_descriptors)} descriptors from cache")
                        return {path: cached_descriptors[path] for path in self.image_paths}
                    else:
                        print("Cache missing some images, recomputing...")
            except Exception as e:
                print(f"Error loading cache: {e}, recomputing...")
        else:
            print("No cache file found, computing descriptors...")
        
        # compute descriptors and save to cache
        print("Computing facial descriptors...")
        descriptors = self.precompute_descriptors()
        
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(descriptors, f)
            print(f"Saved {len(descriptors)} descriptors to {cache_file}")
        except Exception as e:
            print(f"Error saving cache: {e}")
        
        return descriptors
    
    def precompute_descriptors(self):
        """ Precompute facial descriptors for all images to speed up similarity calculations. """
        descriptors = {}
        
        for i, img_path in enumerate(self.image_paths):
            try:
                img = Image.open(img_path)
                img_desc = self.face_recognizer.recognize_faces(np.array(img))
                # store first face descriptor
                if img_desc:
                    descriptors[img_path] = img_desc[0]  
                else:
                    print(f"  No face found in {img_path}")
            except Exception as e:
                print(f"  Error processing {img_path}: {e}")
        
        return descriptors
    
    def get_similar_images(self, image_path, k):
        """ Get k most similar images based on face descriptors. """
        # Get the reference descriptor from precomputed cache
        if image_path not in self.face_descriptors:
            return []
        
        ref_descriptor = self.face_descriptors[image_path]
        
        # Calculate similarity with all other images using precomputed descriptors
        similarities = []
        for img_path, descriptor in self.face_descriptors.items():
            if img_path != image_path:  # Don't compare with itself
                similarity = self.face_recognizer.face_similarity(ref_descriptor, descriptor)
                similarities.append((similarity, img_path))
        
        # Sort by similarity and return top k
        similarities.sort(key=lambda x: x[0])
        return [img[1] for img in similarities[:k]]
    
    def create_image_heap(self):
        """Initialize the heap with images and their weights."""
        # enumerate is done here because otherwise ties are broken alphabetically, want random order
        for i, image_path in enumerate(self.image_paths):
            heapq.heappush(self.image_heap, (0, i, image_path))

In [16]:
def run_face_rating_app():
    """ Run the face rating application. """
    
    faces_dir = "faces"
    if not os.path.exists(faces_dir):
        print(f"Error: '{faces_dir}' directory not found!")
        print("Please create a directory named 'faces'")
        return None
    
    image_paths = [
        os.path.join(faces_dir, f) 
        for f in os.listdir(faces_dir) 
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp"))
    ]
    
    if not image_paths:
        print(f"No images found in '{faces_dir}' directory!")
        print("Please add image files (.jpg, .png, .bmp) to the faces directory.")
        return None
    
    print(f"Found {len(image_paths)} images in '{faces_dir}' directory")
    
    app = JupyterFaceRatingApp(image_paths)
    return app

# Instructions for use
print("Face Rating App - Jupyter Notebook Version")
print("Instructions:")
print("1. Create or load a 'faces' directory with image files")
print("2. Run: app = run_face_rating_app()")
print("3. Run: app.display()")
print("4. Rate faces using the Like/Pass buttons")
print("5. The app learns your preferences and shows similar faces")

Face Rating App - Jupyter Notebook Version
Instructions:
1. Create or load a 'faces' directory with image files
2. Run: app = run_face_rating_app()
3. Run: app.display()
4. Rate faces using the Like/Pass buttons
5. The app learns your preferences and shows similar faces


In [17]:
app = run_face_rating_app()
app.display()

Found 400 images in 'faces' directory
Loading facial descriptors...
Loading descriptors from cache...
Cache missing some images, recomputing...
Computing facial descriptors...
Saved 400 descriptors to face_descriptors_cache.pkl
Loaded descriptors for 400 images
